Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

In [1]:
using LinearAlgebra, Statistics

# using Distributions, Expectations, NLsolve, Roots, Random, Plots, Parameters


using DataFrames
using BenchmarkTools



In [2]:
using NBInclude
@nbinclude("solve_model_cf.ipynb")

#importou a função solve_tree

In [3]:
#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49

data2 = [2.65, 0.69, 0.74];
n2 = 17

#ρ, β, c0, λj
game_parameters = [0.884, 9.84, 0.04, 0.346];

#counterfactual parameters####
D = Float64(1.0)

#simulation of a dip financing where tsym=2
tsym = Int64(2)
dip = Float64(0.05)

additional_cost = dip * D
ac_s = additional_cost
ac_j = additional_cost

counterfactual_parameters = [tsym, additional_cost, ac_s, ac_j];

In [4]:
A1 = solve_tree(data1, game_parameters, counterfactual_parameters);

  3.664230 seconds (44.49 M allocations: 921.648 MiB, 7.84% gc time)
  3.004576 seconds (43.22 M allocations: 858.002 MiB, 3.65% gc time)
  2.917698 seconds (43.22 M allocations: 858.002 MiB, 3.63% gc time)
  2.951713 seconds (43.22 M allocations: 858.002 MiB, 3.72% gc time)
  2.900807 seconds (43.22 M allocations: 858.002 MiB, 3.94% gc time)
  3.546454 seconds (43.22 M allocations: 858.002 MiB, 3.16% gc time)
  7.554649 seconds (43.22 M allocations: 858.002 MiB, 1.80% gc time)
  8.105414 seconds (43.22 M allocations: 858.002 MiB, 1.36% gc time)
  8.742886 seconds (43.22 M allocations: 858.002 MiB, 1.21% gc time)
  9.065311 seconds (43.22 M allocations: 858.002 MiB, 1.21% gc time)
  9.245592 seconds (43.22 M allocations: 858.002 MiB, 1.23% gc time)
  9.228507 seconds (43.22 M allocations: 858.002 MiB, 1.25% gc time)
  9.243170 seconds (43.22 M allocations: 858.002 MiB, 1.20% gc time)
  9.364039 seconds (40.16 M allocations: 811.310 MiB, 1.23% gc time)
 91.419484 seconds (603.34 M alloc

In [5]:
A2 = solve_tree(data2, game_parameters, counterfactual_parameters);

  4.213185 seconds (43.22 M allocations: 858.002 MiB, 3.10% gc time)
  6.212083 seconds (43.22 M allocations: 858.002 MiB, 1.71% gc time)
  7.072276 seconds (43.22 M allocations: 858.002 MiB, 2.07% gc time)
  7.612682 seconds (43.22 M allocations: 858.002 MiB, 1.41% gc time)
  7.908833 seconds (43.22 M allocations: 858.002 MiB, 1.91% gc time)
  8.097839 seconds (43.22 M allocations: 858.002 MiB, 1.32% gc time)
  8.294003 seconds (43.22 M allocations: 858.002 MiB, 1.74% gc time)
  8.444332 seconds (43.22 M allocations: 858.002 MiB, 1.27% gc time)
  8.516969 seconds (43.22 M allocations: 858.002 MiB, 1.69% gc time)
  8.573586 seconds (43.22 M allocations: 858.002 MiB, 1.28% gc time)
  8.742782 seconds (43.22 M allocations: 858.002 MiB, 1.69% gc time)
  8.722839 seconds (40.16 M allocations: 811.310 MiB, 1.24% gc time)
 93.924522 seconds (515.58 M allocations: 10.009 GiB, 1.61% gc time)


Simulation parameters

In [6]:
#θs0, θj0, μ
simulation_parameters = [28, 36, 4.566];

### Criando funções para simular o jogo

In [18]:
#cost function
function Ct(t, c0=c0, c1=c1)
    
    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end




function s_L(t, L, Ds, Dj, c0, c1, ac_s, ac_j)
    SL = min(L - Ct(t, c0, c1) - ac_s, Ds)
    
    return max(SL, zero(0.0))
end

function j_L(t, L, Ds, Dj, c0, c1, ac_s, ac_j)
    JL = min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1, ac_s, ac_j) - ac_j, Dj)
    
    return max(JL, zero(0.0))
end

function proposal(Pkt_array, t, hkt, lmt)
    
    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação
    
    return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]
    
end
    

#maximum value of reorganization each period
function Vt(Vmax, ρ, t, additional_cost)

    if(t <=1)
        return Vmax - additional_cost
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax - additional_cost
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1, ac_s, ac_j, additional_cost)
    
    liq = (m_L(t, L, Ds, Dj, c0, c1, ac_s, ac_j), Vt(Vmax, ρ, t, additional_cost) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)
    
    payoff_liq, answer = findmax(liq)
    
    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)
    
    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)
    
    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next, t, tsym)
    
    if(t<=tsym)
        
        #t <= tsym, there is symmetric information
        
         if(propositor=="s")
            hst = hk_next
            lst = hk_next

            hjt = hm_next
            ljt = hm_next

        else
            hst = hm_next
            lst = hm_next

            hjt = hk_next
            ljt = hk_next
        end
        
    else
        
        #t > tsym, there is asymmetric information


        if(propositor=="s")
            hst = hk_next
            lst = lk_next

            hjt = hm_next
            ljt = lm_next

        else
            hst = hm_next
            lst = lm_next

            hjt = hk_next
            ljt = lk_next
        end
    end

    
    return hst, lst, hjt, ljt
end

        

update_beliefs (generic function with 1 method)

In [19]:
#código para tirar um draw da beta

function draw_beta(hkt, β=game_parameters[2], grid=100)

    u = rand()

    if(hkt == 100)
        
        return 100

    else
        
        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * 100, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)
        
    end
end

        

# @code_warntype draw_beta(10)



draw_beta (generic function with 3 methods)

In [20]:
function choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
    
    if(propositor=="s")
        
        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst
        
        hmt = hjt
        lmt = ljt


        lk_next = hkt

        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)
        
    elseif(propositor=="j")
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt
        
        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    else
        
        println("error: propositor not valid")
        
    end
    
    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end




#test
Hs0 = 30
Hj0 = 28


hst = Hs0
hjt = Hj0

#assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
lst = hst
ljt = hjt




choose_parameters("s", hst, lst, hjt, ljt, A1[1], A1[2], A1[3], A1[4])
    
    

([0.11684475850557874 0.11802112271720203 … 0.7167046035657554 0.7265313000634204; 0.06035727341526065 0.060811721055738684 … 0.6396803302418931 0.6472604885456711; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.11813168239655819 … 0.7167046035657554 0.7265313000634204; 0.0 0.06086400403157448 … 0.6396803302418931 0.6472604885456711; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.7167046035657554 0.7265313000634204; 0.0 0.0 … 0.6396803302418931 0.6472604885456711; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.7167045455028247 0.7265313000634204; 0.0 0.0 … 0.639680272011564 0.6472604885456711; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.7167039340071324 0.7265313000634204; 0.0 0.0 … 0.6396797623839006 0.6472604885456711; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.7265305571738403; 0.0 0.0 … 0.0 0.6472598655381095; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.11680808695647094 0.11793882303866911 … 0.716099159040372 0.7255515411539

In [21]:
function simulate_game(data, game_parameters, simulation_parameters, counterfactual_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
    Vmax = data[1]
    L = data[2]
    Dj = data[3]
    Ds = one(1.0) - Dj
    
    
    c0 = game_parameters[3];
    c1 = Float64(0.0015);

    ρ = game_parameters[1];
    λj = game_parameters[4];

    
    
#     hst = Hs0
#     hjt = Hj0
    
    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt
    
    
    
    
    
    
    
    
    
    result = zeros(5);
    
    #number of observed proposals
    observed_proposals = 0.0
    
    
    #recovering "T+1" from s_W 
    T = size(s_W,1)
    
    
    #counterfactual parameters
    tsym = counterfactual_parameters[1]
    additional_cost = counterfactual_parameters[2]
    ac_s = counterfactual_parameters[3]
    ac_j = counterfactual_parameters[4];
    
    
    
    while(result[1]==0.0 && t < T)
        
        
        u = rand()

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end

        
        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
        
        
        #proposal ####
        policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end
        
        
        
        if(policy==3.0)
            
            observed_proposals += 1.0
    
            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else
                
                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next, t, tsym)
            end

        elseif(policy==2.0)
            
            t+=1
            
            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next, t, tsym)
            
            

        else      
            #(policy==1.0)
            
            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1, ac_s, ac_j, additional_cost)

            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)
            
            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0) #to avoid observed_proposals==0.0
            
        end
        




    end
    
    
    return result
end

        
        
#precisa colocar uma condição IF aqui para caso o jogo vá até o último período?
    

simulate_game (generic function with 7 methods)

In [22]:
# simulate_game(θs0, θj0, μ, λj, s_W, j_W, Pst_array, Pjt_array)

In [23]:
function loop_simulations(S, N, game_parameters, simulation_parameters, counterfactual_parameters, data1, data2, A1, A2, n1)
    #S is the number of simulations
    #N is the number of observations
    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster
    
    
    
    
    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 7)

    #últimas entradas de Results são os valores das dívidas
    
    
    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:5] .= simulate_game(data1, game_parameters, simulation_parameters, counterfactual_parameters, A1[1], A1[2], A1[3], A1[4])
                Results[s,n, end-1] = A1[5]
                Results[s,n, end] = A1[6]
            else
                Results[s,n,1:5] .= simulate_game(data2, game_parameters, simulation_parameters, counterfactual_parameters, A2[1], A2[2], A2[3], A2[4])
                Results[s,n, end-1] = A2[5]
                Results[s,n, end] = A2[6]
            end
            
        end
    end

    return Results
    
end

    
# loop_simulations(40, 75, θs0, θj0, μ, λj);

In [28]:
function simulate_moments(S, N, game_parameters, simulation_parameters, counterfactual_parameters, data1, data2, A1, A2, n1)
    
    Results = loop_simulations(S, N, game_parameters, simulation_parameters, counterfactual_parameters, data1, data2, A1, A2, n1)
    
    
    Moments = zeros(S, 8)
    μ = simulation_parameters[3]
    
    for s in 1:S

            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.Ds = Results[s,:,6];
            DF.Dj = Results[s,:,7];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
            INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = INCOURT
            mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT_REORG
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)

            #6. avg recovery rate for junior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT_REORG
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)


            #7. junior avg fraction gain given incourt REORGANIZATION

            mm7 = INCOURT_REORG
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt REORGANIZATION

            mm8 = INCOURT_REORG
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)



            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]

            #if there are no INCOURT or no PRECOURT cases, it will return NaN
            #thus, we replace it by zero
            Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

        end
    return vec(mean(Moments, dims=1))
end



        
        
#test
S = 40
N = 66


simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, counterfactual_parameters, data1, data2, A1, A2, n1)


    
    

8-element Array{Float64,1}:
 1.1478344071258253
 1.0
 2.051798379685331
 0.7257575757575758
 0.8463598326905132
 0.23401170977257793
 0.4537704251175573
 0.49221984962995907